# Image Restoration using Convolutional Nueral Network 

                                     -Gopalakrishna Adusumilli

**Learning outcomes**


*   Super Resolution
*   SRCNN Network
*   Evaluating the performance of the SRCNN, using popular image quality metrics: **Peak Signal to Noise  Ratio(PSNR)**,  **Mean Squared Error(MSE)** , **the structural similarity(SSIM) index.**
*  Processing of Images using Open CV.
*   Conversion between the RGB,BGR,YCrCb color spaces.
*   Building deep neural networks in keras.
*   Deploy and evaluate the SRCNN network.


##What is Super Resolution ?
Super resolution is the process of upscaling and or improving the details within an image. Often a low resolution image is taken as an input and the same image is upscaled to a higher resolution, which is the output. The details in the high resolution output are filled in where the details are essentially unknown.


##SRCNN Network

SRCNN is a deep convolutional neural network that learns the end-to-end mapping of low resolution to high resolution images. Results, improving the image quality of low resolution images.
![alt text](https://miro.medium.com/max/1050/1*mZJO-i6ImYyXHorv4H1q_Q.png)

















## 1. Importing Packages

In [0]:
#check package versions
import sys
import keras
import cv2
import numpy as np
import matplotlib
import skimage

from keras.models import Sequential
from keras.layers import Conv2D
from keras.optimizers import adam
from skimage.measure import compare_ssim as ssim
from matplotlib import pyplot as plt
import os

#to display pyplot figures in the notebook
%matplotlib inline


##2. Defining Image Quality Metrics

**MSE**: The MSE represents the cumulative squared error between the compressed and the original image

**PSNR**: The PSNR block computes the peak signal-to-noise ratio, in decibels, between two images. This ratio is often used as a quality measurement between the original and a compressed image.
![alt text](https://qph.fs.quoracdn.net/main-qimg-bb07713cb739bfb11abb4f7eb6da3acd.webp)

where M & N in the MSE formula are the number of rows and columns in our image matrix.

**SSIM(Structural Similarity Index)**: SSIM actually measures the perceptual difference between two similar images.
SSIM is based on visible struuctures of the image.

SSIM index will be imported directly from the scikit-image library.


**Objective:**
1. Defining own functions for the PSNR and MSE

2. Wrapping all three of these metrics into a single function that we can call later.

In [0]:
#define a function for peak signal-to-noise ratio (PSNR)

def psnr(target,ref):
  #assume RGB image
  target_data=target.astype(float)
  ref_data=ref.astype(float)
  
  diff=ref_data-target_data
  diff=diff.flatten('c')
  
  rmse=math.sqrt(np.mean(diff**2.))
  
  return 20*math.log10(255. /rmse)


#define function for mean squared error(MSE)

def mse(target,ref):
  #the MSE between the two images is the sum of the squared diffrence between two images
  err=np.sum((target.astype('float')-ref.astype('float'))**2)
  err /=float(target.shape[0]*target.shape[1])
  
  return err

#define function that combines all three image quality metrics
def compare_images(target,ref):
  scores=[]
  scores.append(psnr(target,ref))
  scores.append(mse(target,ref))
  scores.append(ssim(target,ref,multichannel=True))
  
  return scores

  




## 3. Preparing Images

**Dataset**: [Dataset](http://mmlab.ie.cuhk.edu.hk/projects/SRCNN.html). the .ZIP file contain set5 and set14 datasets into a new folder **"Source"**

**Objective**
1. We need to produce low-resolution versions of these images. we can accomplish this by resizing the images,both downwards and upwards,using openCV.

**Approach**
Using Interpolation methods that can be used to resize images, however in this project we will use **Bilinear Transformation**



In [0]:
#prepare degraded images by introducing quality distortions via resizing

def prepare_images(path,factor):
  #loop through the files in the directory
  for file in os.listdir(path):
    #open the file
    img=cv2.imread(path+'/'+file)
    
    #find old and new image dimensions 
    h,w,_=img.shape
    new_height=h/ factor
    new_width=w / factor
    
    #resize the image -down
    img=cv2.resize(img,(w,h),interpolation=cv2.INTER_LINEAR)
    
    #save the image
    print('saving {}'.format(file))
    cv2.imwrite('images/{}'.format(file),img)
prepare_images('source/'2)

## 3. Testing Low-Resolution Images

**Objective**
Calculating Image Metrics on our refereced images and that the degraded images that we just prepared


In [0]:
#test the generated images using the image quality metrics
for file in os.listdir('images/'):
  
  #open target and reference images
  target=cv2.imread('images/{}'.format(file))
  
  ref=cv2.imread('source/{}'.format(file))
  
  #calculate score
  scores=compare_images(target,ref)
  
  #print all three scores with new line charactes (\n)
  print('{}\nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(file, scores[0], scores[1], scores[2]))
  
  

## 4. Building the SRCNN Model

**Objective:**
  Building the SRCNN. iN KERAS, it is simple as adding layers one after the other. 

In [0]:
#define the SRCNN model

def model():
  #define model type
  SRCNN=Sequential()
  
  #add model layers
  
  SRCNN.add(Conv2D(filters=128,kernel_size=(9,9),kernel_initializer='glorot_uniform'
                  activation='relu',padding='valid',use_bias=True,input_shape=(None,None,1)))
  
  SRCNN.add(Conv2D(filters=64,kernel_size=(9,9),kernel_initializer='glorot_uniform'
                  activation='relu',padding='valid',use_bias=True))
  
  SRCNN.add(Conv2D(filters=1,kernel_size=(9,9),kernel_initializer='glorot_uniform'
                  activation='relu',padding='valid'))
  
  
  #define optimizer
  
  adam=Adam(lr=0.0003)
  
  #compile model
  
  SRCNN.compile(optimizer=adam,loss='mean_squared_error',metrics=['mean_squared_error'])
  
  return SRCNN

## 5. Deploying the SRCNN

**Objectives**
1. Preprocessing the Images extensively before using them as inputs to the network. This processing will include cropping and color space conversions.

2. Loading pre-trained model(to save time to train a deep neural network) these weights can be found at the following [Github](https://github.com/MarkPrecursor/SRCNN-keras)

3. Performing single-image super resolution on all our input images. Furhermore, after processing, we can calculate the PSNR,MSE, and SSIM on the images that we produce.

4. Creating subplots to convineantly display the orginal, low resolution, and high resolution images side by side

In [0]:
# define necessary image processing functions

def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img


def shave(image, border):
    img = image[border: -border, border: -border]
    return img
# define main prediction function

def predict(image_path):
    
    # load the srcnn model with weights
    srcnn = model()
    srcnn.load_weights('3051crop_weight_200.h5')
    
    # load the degraded and reference images
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread('source/{}'.format(file))
    
    # preprocess the image with modcrop
    ref = modcrop(ref, 3)
    degraded = modcrop(degraded, 3)
    
    # convert the image to YCrCb - (srcnn trained on Y channel)
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    # create image slice and normalize  
    Y = numpy.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    
    # perform super-resolution with srcnn
    pre = srcnn.predict(Y, batch_size=1)
    
    # post-process output
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    # copy Y channel back to image and convert to BGR
    temp = shave(temp, 6)
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
    
    # remove border from reference and degraged image
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)
    
    # image quality calculations
    scores = []
    scores.append(compare_images(degraded, ref))
    scores.append(compare_images(output, ref))
    
    # return images and scores
    return ref, degraded, output, scores
ref, degraded, output, scores = predict('images/flowers.bmp')

# print all scores for all images
print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))


# display images as subplots
fig, axs = plt.subplots(1, 3, figsize=(20, 8))
axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')
axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
axs[1].set_title('Degraded')
axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
axs[2].set_title('SRCNN')

# remove the x and y ticks
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])


![alt text](https://miro.medium.com/max/1050/1*j9a0kGhWcG8lEDLf5eqcfg.png)

In [0]:
for file in os.listdir('images'):
    
    # perform super-resolution
    ref, degraded, output, scores = predict('images/{}'.format(file))
    
    # display images as subplots
    fig, axs = plt.subplots(1, 3, figsize=(20, 8))
    axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
    axs[0].set_title('Original')
    axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
    axs[1].set_title('Degraded')
    axs[1].set(xlabel = 'PSNR: {}\nMSE: {} \nSSIM: {}'.format(scores[0][0], scores[0][1], scores[0][2]))
    axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    axs[2].set_title('SRCNN')
    axs[2].set(xlabel = 'PSNR: {} \nMSE: {} \nSSIM: {}'.format(scores[1][0], scores[1][1], scores[1][2]))

    # remove the x and y ticks
    for ax in axs:
        ax.set_xticks([])
        ax.set_yticks([])
      
    print('Saving {}'.format(file))
    fig.savefig('output/{}.png'.format(os.path.splitext(file)[0])) 
    plt.close()


![alt text](https://miro.medium.com/max/1050/1*8lpeTi2p_F7AhE2o_7tJ9Q.png)